In [78]:
import numpy as np
import matplotlib.pyplot as plt

In [79]:
POINTS = 500

In [80]:
def f(x):
    return np.sin(2*x) * np.sin(x**2/np.pi)

In [81]:
def monomial_base(i):
    return lambda x: x**i

In [82]:
def trigonomerical_base(i):
    k = (i + 1) // 2
    if i % 2 == 1:
        return lambda x: np.sin(k * (x * 2 / 3 + np.pi))
    return lambda x: np.cos(k * (x * 2 / 3 + np.pi))

In [83]:
def calculate_matrix_elem(row, col, x_arr, wage_fun, base_generator):
    fi_k = base_generator(row)
    fi_j = base_generator(col)
    return np.sum(wage_fun(x_arr) * fi_k(x_arr) * fi_j(x_arr))

In [84]:
def calculate_vector_elem(row, x_arr, y_arr, wage_fun, base_generator):
    fi_k = base_generator(row)
    return np.sum(wage_fun(x_arr) * y_arr * fi_k(x_arr))

In [85]:
def generate_eq_sys_matrix(m, x_arr, wage_fun, base_generator):
    result = np.empty((m,m))
    for i in range(m):
        for j in range(m):
            result[i][j] = calculate_matrix_elem(i, j, x_arr, wage_fun, base_generator)
    return result

In [86]:
def generate_eq_sys_dependent_vector(m, x_arr, y_arr, wage_fun, base_generator):
    result = np.empty((m,))
    for i in range(m):
        result[i] = calculate_vector_elem(i, x_arr, y_arr, wage_fun, base_generator)
    return result

In [87]:
def compute_a_coeff(m, x_arr, y_arr, wage_fun, base_generator):
    g = generate_eq_sys_matrix(m, x_arr, wage_fun, base_generator)
    b = generate_eq_sys_dependent_vector(m, x_arr, y_arr, wage_fun, base_generator)
    return np.linalg.solve(g, b)

In [88]:
def print_errors(y_print, p_print):
    print('Błąd interpolacji(norma euklidesowa):', np.linalg.norm(y_print-p_print) / POINTS)
    print('Błąd interpolacji(norma maksimum):', np.linalg.norm(y_print-p_print, ord=np.inf))

In [89]:
def print_plots(x_approx, x_print, y_approx, y_print, p_print):
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.plot(x_print, p_print, label="W(x)")
    ax.plot(x_print, y_print, label="f(x)")
    ax.plot(x_approx, y_approx, '*', label='Węzły aproksymacji')
    ax.set_title('elo')
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.legend()
    plt.show()

In [90]:
def calculate_polynomial_value(x, polynomial, m):
    values = np.empty((m,))
    for i in range(m):
        monomial = polynomial[i]
        values[i] = monomial(x)
    return np.sum(values)

In [91]:
def evaluate_fun(fun, x):
    return fun(x)

In [92]:
def evaluate_approximation_polynomial(x, a_coeff, base):
    #if base == trigonomerical_base:
    #    x = np.pi + x * 2 / 3
    m = len(a_coeff)
    values = [] #np.empty((m,))
    for i in range(m):
        monomial = base(i)
        value = a_coeff[i] * monomial(x)
        values.append(value) # = evaluate_fun(monomial, x)
    return sum(values)

In [93]:
def approximate(m, x_print, x_approx, y_approx, wage_fun, base_generator):
    a = compute_a_coeff(m, x_approx, y_approx, wage_fun, base_generator)
    return evaluate_approximation_polynomial(x_print, a, base_generator)

In [102]:
def exercise(a, b, n, m, wage, base):
    if base == trigonomerical_base:
        plot_name = "Liczba f. trygonometrycznych (m): {0:d}, liczba węzłów (n): {1:d}".format(m, n)
    else:
        plot_name = "Stopień wielomianu (m): {0:d}, liczba węzłów (n): {1:d}".format(m, n)
    m = m + 1
    print(plot_name)
    x_approx = np.linspace(a, b, n)
    x_print = np.linspace(a, b, POINTS)
    y_approx = f(x_approx)
    y_print = f(x_print)
    p_print = approximate(m, x_print, x_approx, y_approx, wage_fun=wage, base_generator=base)
    eukl_n, max_n = calculate_errors(y_print, p_print)
    print_errors(eukl_n, max_n)
    print_plots(x_approx, x_print, y_approx, y_print, p_print, plot_name)
    return eukl_n, max_n

In [103]:
exercise(-np.pi, 2 * np.pi, 50, 5, monomial_base)

TypeError: exercise() missing 1 required positional argument: 'base'

In [104]:
exercise(-np.pi, 2 * np.pi, 50, 15, trigonomerical_base)

TypeError: exercise() missing 1 required positional argument: 'base'

In [106]:
def read_data():
    polynomial_errors = exercise_1()
    # cubic = exercise_2()
    polynomial_data = {}
    outside = [] #list(polynomial_errors.keys())
    for key in polynomial_errors[2]['Eukl'].keys():
        outside += list(polynomial_errors.keys())
    outside.sort()
    inside = []
    for key in polynomial_errors.keys():
        inside += polynomial_errors[key]['Eukl'].keys()
    hier_index = list(zip(outside, inside))
    hier_index = pd.MultiIndex.from_tuples(hier_index)
    print(outside, inside)
    print(list(zip(outside, inside)))
    print(hier_index)
    #for key in polynomial_errors.keys():
    #    polynomial_data[key] = pd.DataFrame(polynomial_errors[key])
    data['Polynomial'] = pd.DataFrame(polynomial_data, index=hier_index)
    # data['Cubic'] = pd.DataFrame(cubic)

In [107]:
def exercise_polynomial(m):
    a = -np.pi
    b = 2 * np.pi
    eukl_errors = {}
    max_errors = {}
    for n in range(20, 61, 20):
        eukl_n, max_n = exercise(a, b, n, m, lambda x: 1, monomial_base)
        eukl_errors[n] = eukl_n
        max_errors[n] = max_n
    polynomial_errors = {'Eukl': eukl_errors, 'Max': max_errors}
    return polynomial_errors


def exercise_trigonometrical(m):
    a = -np.pi
    b = 2 * np.pi
    eukl_errors = {}
    max_errors = {}
    for n in range(30, 101, 30):
        eukl_n, max_n = exercise(a, b, n, m, lambda x: 1, trigonometrical_base)
        eukl_errors[n] = eukl_n
        max_errors[n] = max_n
    polynomial_errors = {'Eukl': eukl_errors, 'Max': max_errors}
    return polynomial_errors


def exercise_1():
    errors = {}
    for m in range(1, 11):
        errors[m] = exercise_polynomial(m)
    return errors


def exercise_2():
    errors = {}
    for m in range(1, 26):
        errors[m] = exercise_trigonometrical(m)
    return errors

In [108]:
data = {}

In [109]:
read_data()

Stopień wielomianu (m): 1, liczba węzłów (n): 20


NameError: name 'calculate_errors' is not defined